In [70]:
import pandas as pd
import numpy as np

In [71]:
df = pd.read_csv("data/train.csv")

In [72]:
df = df.replace(r'^\s*$', np.nan, regex=True)
#Filling with mode for the other columns
df['STATE'].fillna(df['STATE'].mode()[0], inplace=True)
df['OCCUPATION'].fillna('U1', inplace=True)
df['INCOME_GROUP'].fillna(df['INCOME_GROUP'].mode()[0], inplace=True)
df['LOYALTY_PROGRAM'].fillna(df['LOYALTY_PROGRAM'].mode()[0], inplace=True)
#df.describe()

In [73]:
from scipy import stats

In [74]:
df = df[(np.abs(stats.zscore(df['PAST_PURCHASE'])) < 3)]

In [75]:
df

,ID,STATE,AGE,OCCUPATION,INCOME_GROUP,CUSTOMER_SINCE,LOYALTY_PROGRAM,PAST_PURCHASE,PURCHASE
0,1,Uttar Pradesh,28,U1,Low,2018,No,4937,No
1,2,Madhya Pradesh,27,Salaried,Low,2018,No,10834,No
2,3,West Bengal,62,U1,High,2013,No,8610,Yes
3,4,Tamil Nadu,27,Salaried,Low,2018,No,14374,Yes
4,5,Uttar Pradesh,40,Self employed,High,2015,No,13158,No
...,...,...,...,...,...,...,...,...,...
149995,149996,Assam,29,Salaried,Low,2017,No,6241,No
149996,149997,Andhra Pradesh,53,Self employed,Medium,2010,No,29447,No
149997,149998,Odisha,31,U1,Low,2018,No,4856,No
149998,149999,Andhra Pradesh,54,Self employed,High,2010,Yes,10031,Yes


In [76]:
#Dropping the ID column 
df = df.drop(['ID'], axis = 1)

In [77]:
#df['LOYALTY_PROGRAM'] = df['LOYALTY_PROGRAM'].map({'Yes': 1, 'No': 0})
df['PURCHASE'] = df['PURCHASE'].map({'Yes': 1, 'No': 0})

In [78]:
from sklearn.model_selection import train_test_split

In [79]:
#Split the dataset into 70(train) - 30(test)
X = df.drop(['PURCHASE'], axis=1)
y = df['PURCHASE']

In [80]:
import category_encoders as ce

In [81]:
# Define catboost encoder
cbe_encoder = ce.cat_boost.CatBoostEncoder()
  
# Fit encoder and transform the features
cbe_encoder.fit(X, y)
train_cbe = cbe_encoder.transform(X)

In [82]:
X_train, X_test, y_train, y_test = train_test_split(train_cbe, y, test_size = 0.3, random_state = 0)

In [83]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from xgboost import XGBClassifier
# fit model no training data
model = XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("F1 Score: ",f1_score(y_test, predictions))

C:\Users\vishn\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:59:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 78.87%
F1 Score:  0.3201050481470675


In [67]:
from catboost import CatBoostClassifier
# fit model no training data
model = CatBoostClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("F1 Score: ",f1_score(y_test, predictions))

Learning rate set to 0.074517
0:	learn: 0.6326798	total: 19.8ms	remaining: 19.8s
1:	learn: 0.5821793	total: 38.8ms	remaining: 19.4s
2:	learn: 0.5416806	total: 58.3ms	remaining: 19.4s
3:	learn: 0.5086178	total: 75ms	remaining: 18.7s
4:	learn: 0.4815715	total: 93.7ms	remaining: 18.6s
5:	learn: 0.4601731	total: 110ms	remaining: 18.2s
6:	learn: 0.4433561	total: 125ms	remaining: 17.7s
7:	learn: 0.4282056	total: 142ms	remaining: 17.7s
8:	learn: 0.4157943	total: 161ms	remaining: 17.7s
9:	learn: 0.4049774	total: 180ms	remaining: 17.9s
10:	learn: 0.3971651	total: 198ms	remaining: 17.8s
11:	learn: 0.3904012	total: 219ms	remaining: 18.1s
12:	learn: 0.3844223	total: 238ms	remaining: 18.1s
13:	learn: 0.3797177	total: 257ms	remaining: 18.1s
14:	learn: 0.3756616	total: 278ms	remaining: 18.2s
15:	learn: 0.3723090	total: 297ms	remaining: 18.3s
16:	learn: 0.3689526	total: 317ms	remaining: 18.3s
17:	learn: 0.3662988	total: 332ms	remaining: 18.1s
18:	learn: 0.3643450	total: 350ms	remaining: 18.1s
19:	lear

In [68]:
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("F1 Score: ",f1_score(y_test, predictions))

Accuracy: 85.87%
F1 Score:  0.6366180758017492


In [69]:
from lightgbm import LGBMClassifier
# fit model no training data
model = LGBMClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("F1 Score: ",f1_score(y_test, predictions))

Accuracy: 85.89%
F1 Score:  0.6306733906852566


As we can see, we get pretty much the same F1 score even if we remove the outliers. 